In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch
from tqdm import tqdm

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from surface_vae_trainer import SurfaceVAETrainer
from dupire_pinn_trainer import DupirePINNTrainer
from adaptive_latent_evolutionary_optimization import AdaptiveEvolutionaryLatentOptimization

In [2]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [3]:
n_roots = 350
# n_roots = 10
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

prior_covariance_matrix = smoothness_prior.prior_covariance()
prior_eigenvalues = np.sort(np.linalg.eigvalsh(prior_covariance_matrix))[::-1].copy()

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(1000)

# Loop through the sampled coefficients 
sampled_volatilities = []
for coefficients in sampled_surface_coefficients:
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = [
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in product_grid
    ]
    sampled_volatilities.extend(surface_volatilities)

hidden_dim = 64
n_layers = 8
batch_size = 100
pde_loss_coefficient = 650.0
maturity_zero_loss_coefficient = 800.0
strike_zero_loss_coefficient = 40.0
strike_infinity_loss_coefficient = 200.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 200
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
torch.manual_seed(0)
pinn_trainer = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

pinn_trainer.load_model()

pinn_trainer.dupire_price_prediction_loss(
    torch.tensor(sampled_surface_coefficients, device=device, dtype=torch.float32),
    call_option_dataset["Call Option Price"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
)
print()    

PINN Model loaded from models/pinn_model.pth.



In [5]:
latent_dim = 40  # Latent dimension
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

torch.manual_seed(2)
vae_trainer = SurfaceVAETrainer(
    latent_dim=latent_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    data_dim=data_dim,
    latent_diagonal=latent_diagonal,
    batch_size=batch_size,
    beta=beta_,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    device=device,
)

vae_trainer.load_model()

VAE Model loaded from models/vae_model.pth.


In [ ]:
pinn_trainer.fine_tune_learning_rate = 1e-4
pinn_trainer.fine_tune_epochs = 2
vae_trainer.fine_tune_learning_rate = 1e-4
vae_trainer.fine_tune_epochs = 6


population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
n_cycles = 10
torch.manual_seed(2)
pinn_trainer.load_model()
vae_trainer.load_model()
# Initialize the AdaptiveEvolutionaryLatentOptimization
adelo = AdaptiveEvolutionaryLatentOptimization(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    n_cycles=n_cycles
)

# Run the optimization process
adelo.run_cycle()
adelo.plot_evolutions()

In [ ]:
pinn_trainer.fine_tune_learning_rate = 1e-4
pinn_trainer.fine_tune_epochs = 20
vae_trainer.fine_tune_learning_rate = 1e-4
vae_trainer.fine_tune_epochs = 60


population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
n_cycles = 10
torch.manual_seed(2)
pinn_trainer.load_model()
vae_trainer.load_model()
# Initialize the AdaptiveEvolutionaryLatentOptimization
adelo = AdaptiveEvolutionaryLatentOptimization(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    n_cycles=n_cycles
)

# Run the optimization process
adelo.run_cycle()
adelo.plot_evolutions()

In [ ]:
pinn_trainer.fine_tune_learning_rate = 1e-4
pinn_trainer.fine_tune_epochs = 100
vae_trainer.fine_tune_learning_rate = 1e-4
vae_trainer.fine_tune_epochs = 300


population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
n_cycles = 10
torch.manual_seed(2)
pinn_trainer.load_model()
vae_trainer.load_model()
# Initialize the AdaptiveEvolutionaryLatentOptimization
adelo = AdaptiveEvolutionaryLatentOptimization(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    n_cycles=n_cycles
)

# Run the optimization process
adelo.run_cycle()
adelo.plot_evolutions()

In [ ]:
pinn_trainer.fine_tune_learning_rate = 1e-3
pinn_trainer.fine_tune_epochs = 2
vae_trainer.fine_tune_learning_rate = 1e-3
vae_trainer.fine_tune_epochs = 6


population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
n_cycles = 10
torch.manual_seed(2)
pinn_trainer.load_model()
vae_trainer.load_model()
# Initialize the AdaptiveEvolutionaryLatentOptimization
adelo = AdaptiveEvolutionaryLatentOptimization(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    n_cycles=n_cycles
)

# Run the optimization process
adelo.run_cycle()
adelo.plot_evolutions()

In [6]:
pinn_trainer.fine_tune_learning_rate = 1e-3
pinn_trainer.fine_tune_epochs = 20
vae_trainer.fine_tune_learning_rate = 1e-3
vae_trainer.fine_tune_epochs = 60


population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
n_cycles = 10
torch.manual_seed(2)
pinn_trainer.load_model()
vae_trainer.load_model()
# Initialize the AdaptiveEvolutionaryLatentOptimization
adelo = AdaptiveEvolutionaryLatentOptimization(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    n_cycles=n_cycles
)

# Run the optimization process
adelo.run_cycle()
adelo.plot_evolutions()

PINN Model loaded from models/pinn_model.pth.
VAE Model loaded from models/vae_model.pth.
Starting Cycle 1/10
Running Evolutionary Optimization...


100%|██████████| 500/500 [00:03<00:00, 137.51it/s]


Cycle 1/10 Completed.

Starting Cycle 2/10
Fine-Tuning VAE and PINN...


100%|██████████| 20/20 [00:04<00:00,  4.74it/s]


Running Evolutionary Optimization...


...

Cycle 9/10 Completed.

Starting Cycle 10/10
Fine-Tuning VAE and PINN...


100%|██████████| 20/20 [00:04<00:00,  4.76it/s]


Running Evolutionary Optimization...


100%|██████████| 500/500 [00:03<00:00, 141.59it/s]


Cycle 10/10 Completed.



In [ ]:
pinn_trainer.fine_tune_learning_rate = 1e-3
pinn_trainer.fine_tune_epochs = 100
vae_trainer.fine_tune_learning_rate = 1e-3
vae_trainer.fine_tune_epochs = 300


population_size = 100 # Population size
mutation_strength = 100.0  # Mutation strength for Gaussian noise
selection_pressure_parameter = 0.7  # Selection pressure parameter (eta)
n_generations = 500  # Number of generations for the evolutionary process
n_cycles = 10
torch.manual_seed(2)
pinn_trainer.load_model()
vae_trainer.load_model()
# Initialize the AdaptiveEvolutionaryLatentOptimization
adelo = AdaptiveEvolutionaryLatentOptimization(
    vae_trainer=vae_trainer,
    pinn_trainer=pinn_trainer,
    latent_diagonal=latent_diagonal,
    population_size=population_size,
    mutation_strength=mutation_strength,
    selection_pressure_parameter=selection_pressure_parameter,
    n_generations=n_generations,
    n_cycles=n_cycles
)

# Run the optimization process
adelo.run_cycle()
adelo.plot_evolutions()